In [1]:
import sys
sys.path.append('./../')
from xd import *
from utils import *
import esutil
import healpy as hp
from systematics import *
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import esutil
import healpy as hp
from systematics import *

import yaml
import argparse

In [3]:
from systematics_module.corr import *

In [4]:
def priorCut_test(data):
    print 'CHECK input catalog has only galaxies'
    ## Should add MODEST_CLASS cut later.
    modelmag_g_des = data['MAG_DETMODEL_G']
    modelmag_r_des = data['MAG_DETMODEL_R']
    modelmag_i_des = data['MAG_DETMODEL_I']
    cmodelmag_g_des = data['MAG_MODEL_G']
    cmodelmag_r_des = data['MAG_MODEL_R']
    cmodelmag_i_des = data['MAG_MODEL_I']
    magauto_des = data['MAG_AUTO_I']

    cut = (((cmodelmag_r_des > 17) & (cmodelmag_r_des <24)) &
           ((cmodelmag_i_des > 17) & (cmodelmag_i_des <24)) &
           ((cmodelmag_g_des > 17) & (cmodelmag_g_des <24)) &
           ((modelmag_r_des - modelmag_i_des ) < 1.5 ) & # 10122 (95%)
           ((modelmag_r_des - modelmag_i_des ) > 0. ) & # 10120 (95%)
           ((modelmag_g_des - modelmag_r_des ) > 0. ) & # 10118 (95%)
           ((modelmag_g_des - modelmag_r_des ) < 2.5 ) & # 10122 (95%)
           (magauto_des < 21. ) #&  10124 (95%)
        )
    return cut

In [5]:
def prepare_cmass_sample(params):

    cmass = io.getSGCCMASSphotoObjcat()
    print 'num of cmass in sgc region', cmass.size
    print '\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------'
    cmass = Cuts.keepGoodRegion(cmass)
    print 'num of cmass after des veto', cmass.size

    return cmass

In [6]:
def train_st82(params, param_file):

    # calling params
    output_dir = params['output_dir']
    #cmass_fraction = params['cmass_fraction']
    train_sample_filename = params['train_sample']
    cmass_pickle = output_dir + params['cmass_pickle']
    no_pickle = output_dir + params['no_pickle']
    #out_catname = params['out_catname']
    #out_resampled_cat = params['out_resampled_cat']
    #input_path = params['input_cat_dir']
    #input_keyword = params['input_cat_keyword']


    # calling map --------------------
    GoldMask = callingEliGoldMask()
    GoldMask_st82 = GoldMask[ GoldMask['DEC'] > -3.0 ]
    GoldMask_spt = GoldMask[ GoldMask['DEC'] < -3.0 ]

    pixarea = hp.nside2pixarea( 4096, degrees = True)
    sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
    st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
    SPTMaparea = pixarea * sptnpix
    ST82Maparea = pixarea * st82npix


    # calling stripe82 gold catalogue -----------------------------
    path = '/n/des/lee.5922/data/gold_cat/'
    columns = ['FLAGS_GOLD', 'FLAGS_BADREGION', 'MAG_MODEL_G', 'MAG_MODEL_R', 'MAG_MODEL_I', 'MAG_MODEL_Z',\
               'MAG_DETMODEL_G', 'MAG_DETMODEL_R', 'MAG_DETMODEL_I', 'MAG_DETMODEL_Z', 'MAGERR_DETMODEL_G',\
               'MAGERR_DETMODEL_R', 'MAGERR_DETMODEL_I', 'MAGERR_DETMODEL_Z', 'MAGERR_MODEL_G', 'MAGERR_MODEL_R',\
               'MAGERR_MODEL_I', 'MAGERR_MODEL_Z', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'RA',\
               'DEC', 'COADD_OBJECTS_ID', 'MODEST_CLASS', 'HPIX', 'DESDM_ZP']

    gold_st82 = io.SearchAndCallFits(path = path, columns = columns, keyword = 'Y1A1_GOLD_STRIPE82_v2')
    gold_st82 = gold_st82[gold_st82['MODEST_CLASS'] == 1]
    gold_st82 = Cuts.keepGoodRegion(gold_st82)

    # flags and color cut
    mask_all = (gold_st82['FLAGS_GOLD'] == 0 )&(priorCut_test(gold_st82))
    gold_st82 = gold_st82[mask_all]


    # calling BOSS cmass and applying dmass goodregion mask ----------------------------
    #cmass = io.getSGCCMASSphotoObjcat()
    train_sample = esutil.io.read(train_sample_filename)
    print 'total num of train', train_sample.size
    print '\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------'
    train_sample = Cuts.keepGoodRegion(train_sample)
    print 'num of train_sample after des veto', train_sample.size

    print '\n--------------------------------\n matching catalogues\n--------------------------------'

    # find cmass in des_gold side --------------------
    mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82['RA'], \
                                         gold_st82['DEC'],2./3600, maxmatch=1)
    cmass_mask = np.zeros(gold_st82.size, dtype=bool)
    cmass_mask[mg2] = 1
    clean_cmass_data_des, nocmass = gold_st82[cmass_mask], gold_st82[~cmass_mask]


    print 'num of cmass in des side', clean_cmass_data_des.size, '({:0.0f}%)'.format(clean_cmass_data_des.size*1./train_sample.size * 100)
    print 'num of non-cmass in des side ', nocmass.size

    if params['random_sampling'] :
        random_sampling_ind = np.random.choice(np.arange(nocmass.size), size = nocmass.size/10)
        nocmass = nocmass[random_sampling_ind]
        print 'num of randomly sampled non-cmass ', nocmass.size

    cmass_fraction = clean_cmass_data_des.size *1./gold_st82.size
    print 'cmass_fraction', cmass_fraction
    f = open(output_dir+'cmassfrac', 'w')
    f.write('{0:.10f}'.format(cmass_fraction))

    gold_st82 = None  # initialize called catalogs to save memory
    
    #params['cmass_fraction'] = cmass_fraction
    print '\n--------------------------------\n Extreme deconvolution fitting\n--------------------------------'
    # Fitting ----------------------------------------------

    n_cmass, n_no = None, None
    if 'n_cmass' in params : n_cmass = params['n_cmass']
    if 'n_no' in params : n_no = params['n_no']
    clf_cmass = XD_fitting( data = clean_cmass_data_des, pickleFileName = cmass_pickle, n_cl = n_cmass )
    clf_no = XD_fitting( data = nocmass, pickleFileName = no_pickle , n_cl = n_no)


    print '\n--------------------------\n Fitting End\n---------------------------'

In [7]:
pwd

u'/n/des/lee.5922/Dropbox/repositories/CMASS/code/notebook'

In [8]:
param_file = '../yaml/config_xd_bovy.yaml'
params = yaml.load(open(param_file))
output_dir = params['output_dir']

In [9]:
# calling params
output_dir = params['output_dir']
#cmass_fraction = params['cmass_fraction']
train_sample_filename = params['train_sample']
cmass_pickle = output_dir + params['cmass_pickle']
no_pickle = output_dir + params['no_pickle']
#out_catname = params['out_catname']
#out_resampled_cat = params['out_resampled_cat']
#input_path = params['input_cat_dir']
#input_keyword = params['input_cat_keyword']


# calling map --------------------
GoldMask = callingEliGoldMask()
GoldMask_st82 = GoldMask[ GoldMask['DEC'] > -3.0 ]
GoldMask_spt = GoldMask[ GoldMask['DEC'] < -3.0 ]

pixarea = hp.nside2pixarea( 4096, degrees = True)
sptnpix = GoldMask_spt['PIXEL'].size #hp.get_map_size( GoldMask_spt['PIXEL'] )
st82npix =  GoldMask_st82['PIXEL'].size # hp.get_map_size( GoldMask_st82 )
SPTMaparea = pixarea * sptnpix
ST82Maparea = pixarea * st82npix


# calling stripe82 gold catalogue -----------------------------
path = '/n/des/lee.5922/data/gold_cat/'
columns = ['FLAGS_GOLD', 'FLAGS_BADREGION', 'MAG_MODEL_G', 'MAG_MODEL_R', 'MAG_MODEL_I', 'MAG_MODEL_Z',\
           'MAG_DETMODEL_G', 'MAG_DETMODEL_R', 'MAG_DETMODEL_I', 'MAG_DETMODEL_Z', 'MAGERR_DETMODEL_G',\
           'MAGERR_DETMODEL_R', 'MAGERR_DETMODEL_I', 'MAGERR_DETMODEL_Z', 'MAGERR_MODEL_G', 'MAGERR_MODEL_R',\
           'MAGERR_MODEL_I', 'MAGERR_MODEL_Z', 'MAG_AUTO_G', 'MAG_AUTO_R', 'MAG_AUTO_I', 'MAG_AUTO_Z', 'RA',\
           'DEC', 'COADD_OBJECTS_ID', 'MODEST_CLASS', 'HPIX', 'DESDM_ZP']

gold_st82 = io.SearchAndCallFits(path = path, columns = columns, keyword = 'Y1A1_GOLD_STRIPE82_v2')
gold_st82 = gold_st82[gold_st82['MODEST_CLASS'] == 1]
gold_st82 = Cuts.keepGoodRegion(gold_st82)

# flags and color cut
mask_all = (gold_st82['FLAGS_GOLD'] == 0 )&(priorCut_test(gold_st82))
gold_st82 = gold_st82[mask_all]


--------------------------------
 calling catalog
--------------------------------
Y1A1_GOLD_STRIPE82_v2_000001.fits
Y1A1_GOLD_STRIPE82_v2_000002.fits
Y1A1_GOLD_STRIPE82_v2_000003.fits
Y1A1_GOLD_STRIPE82_v2_000004.fits
Y1A1_GOLD_STRIPE82_v2_000005.fits
Y1A1_GOLD_STRIPE82_v2_000006.fits
Y1A1_GOLD_STRIPE82_v2_000007.fits
Y1A1_GOLD_STRIPE82_v2_000008.fits
reading 1/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000001.fits
reading 2/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000002.fits
reading 3/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000003.fits
reading 4/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000004.fits
reading 5/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000005.fits
reading 6/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000006.fits
reading 7/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000007.fits
reading 8/8 /n/des/lee.5922/data/gold_cat/Y1A1_GOLD_STRIPE82_v2_000008.fits
CHECK input catalog has only galaxie

In [10]:

# calling BOSS cmass and applying dmass goodregion mask ----------------------------
#cmass = io.getSGCCMASSphotoObjcat()
train_sample = esutil.io.read(train_sample_filename)
print 'total num of train', train_sample.size
print '\n--------------------------------\n applying DES veto mask to CMASS\n--------------------------------'
train_sample = Cuts.keepGoodRegion(train_sample)
print 'num of train_sample after des veto', train_sample.size

print '\n--------------------------------\n matching catalogues\n--------------------------------'

# find cmass in des_gold side --------------------
mg1, mg2, _ = esutil.htm.HTM(10).match(train_sample['RA'], train_sample['DEC'], gold_st82['RA'], \
                                     gold_st82['DEC'],2./3600, maxmatch=1)

train_sample = train_sample[mg1]
clean_cmass_data_des = gold_st82[mg2]

cmass_mask = np.zeros(gold_st82.size, dtype=bool)
cmass_mask[mg2] = 1
no_cmass = gold_st82[~cmass_mask]
#clean_cmass_data_des, nocmass = gold_st82[cmass_mask], gold_st82[~cmass_mask]


total num of train 230831

--------------------------------
 applying DES veto mask to CMASS
--------------------------------
num of train_sample after des veto 11908

--------------------------------
 matching catalogues
--------------------------------


In [69]:
w_sgc = train_sample['WEIGHT_SYSTOT']*( train_sample['WEIGHT_CP'] + train_sample['WEIGHT_NOZ'] - 1. )
clean_cmass_data_des['CMASS_WEIGHT'] = w_sgc
#clean_cmass_data_des = appendColumn(clean_cmass_data_des, name = 'CMASS_WEIGHT', value = w_sgc)

In [12]:
if params['random_sampling'] :
    random_sampling_ind = np.random.choice(np.arange(no_cmass.size), size = no_cmass.size/10)
    no_cmass = no_cmass[random_sampling_ind]
    print 'num of randomly sampled non-cmass ', no_cmass.size

cmass_fraction = clean_cmass_data_des.size *1./gold_st82.size
print 'cmass_fraction', cmass_fraction
f = open(output_dir+'cmassfrac', 'w')
f.write('{0:.10f}'.format(cmass_fraction))

gold_st82 = None  # initialize called catalogs to save memory

#params['cmass_fraction'] = cmass_fraction
print '\n--------------------------------\n Extreme deconvolution fitting\n--------------------------------'
# Fitting ----------------------------------------------

num of randomly sampled non-cmass  69673
cmass_fraction 0.0165126166668

--------------------------------
 Extreme deconvolution fitting
--------------------------------


In [65]:
def extreme_fitting( cat, n_comp = 20, xmean=None, xamp=None, xcovar=None, pickle_name = '../output/test/pickle.pkl', log='../xd_log/log', weight=None ):
    from extreme_deconvolution import extreme_deconvolution
    from xd import XDGMM
    import pickle
    
    ydata, ycovar = mixing_color(cat)
    if weight is True : weight = cat['CMASS_WEIGHT']

    if xmean is None : 
        from sklearn.mixture import GMM
        print 'initial guess'
        gmm = GMM(n_comp, n_iter= 10, covariance_type='full',
                          random_state=1).fit(ydata)
        xmean = gmm.means_
        xamp = gmm.weights_
        xcovar = gmm.covars_

        #return xmean, xamp, xcovar

    print 'fitting started'
    
    l = extreme_deconvolution(ydata, ycovar, xamp, xmean, xcovar, projection=None, weight=weight, \
                          fixamp=None, fixmean=None, fixcovar=None, tol=1e-10, maxiter=1000000, \
                          w=0.0, logfile=log, splitnmerge=0, maxsnm=False, likeonly=False, logweight=False)
 
    
    data = XDGMM(n_comp, tol= 1e-6)
    data.V = xcovar
    data.mu = xmean
    data.alpha = xamp
    
    clf = {'retval' : data }
    output = open( pickle_name, 'wb')
    pickle.dump( clf, output )
    print 'pickle saved ', pickle_name
    
    #return clf

In [ ]:
def loadpickle(filename)
    import pickle
    f = open(filename)
    pickle = pickle.load(f)
    clf = pickle['retval']
    xamp = clf.alpha
    xmean = clf.mu
    xcovar = clf.V
    ncomp = clf.n_components
    return ncomp, xamp, xmean, xcovar

In [17]:
xmean, xamp, xcovar = extreme_fitting(no_cmass, n_comp = ncomp, \
                                      xamp=xamp, xmean=xmean, xcovar=xcovar, \
                                      log='../xd_log/no_cmass')

In [70]:
import pickle
f = open('../output/n2/gold_st82_XD_cmass.pkl')
pickle = pickle.load(f)
clf = pickle['retval']
xamp = clf.alpha
xmean = clf.mu
xcovar = clf.V
ncomp = clf.n_components

extreme_fitting(clean_cmass_data_des, n_comp = ncomp, weight=True,\
              xamp=xamp, xmean=xmean, xcovar=xcovar, \
              log='../xd_log/cmass', pickle_name='../output/test/gold_st82_XD_cmass_sys_cp_nz_weight.pkl')

fitting started
pickle saved  ../output/test/gold_st82_XD_cmass_sys_cp_nz_weight.pkl


In [62]:
clf_cmass = XD_fitting(None, pickleFileName='../output/test/pickle.pkl')